# Capítulo 5 - Classificação dos dados

## Bibliotecas básicas e outras inicializações

In [1]:
import warnings
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

warnings.filterwarnings('ignore')

## Carregamento dos dados

In [2]:
dados = pd.read_csv('./datasets/mammographic_masses.data', 
                    names=['bi_rads', 'idade', 'forma_da_massa', 'contorno_da_massa', 'densidade_da_massa', 'severidade'], 
                    na_values='?')

dados['severidade'] = dados['severidade'].map({0 : 'benigno', 1: 'maligno'})
print(f'Linhas: {dados.shape[0]} | Colunas: {dados.shape[1]}')
dados.head()

Linhas: 961 | Colunas: 6


,bi_rads,idade,forma_da_massa,contorno_da_massa,densidade_da_massa,severidade
0,5.0,67.0,3.0,5.0,3.0,maligno
1,4.0,43.0,1.0,1.0,NaN,maligno
2,5.0,58.0,4.0,5.0,3.0,maligno
3,4.0,28.0,1.0,1.0,3.0,benigno
4,5.0,74.0,1.0,5.0,NaN,maligno


## Dicionário dos dados

**bi_rads**: sistema de clasificação adotado para estimar qual a chance de determinada imagem da mamografia ser câncer.{0-6}(float)

**idade**: idade do paciente em anos.(float)

**forma_da_massa**: {redonda=1, ovular=2, lobular=3, irregular=4}(float)

**contorno_da_massa**: {circunscrita=1, microlobulada=2, obscura=3, mal definida=4, especulada=5}(float)

**densidade_da_massa**: {alta=1, iso=2, baixa=3, gordurosa=4}(float)

**severidade**: {benigno=0, maligno=1}(objeto)

## Informação dos dados

In [3]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 961 entries, 0 to 960
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   bi_rads             959 non-null    float64
 1   idade               956 non-null    float64
 2   forma_da_massa      930 non-null    float64
 3   contorno_da_massa   913 non-null    float64
 4   densidade_da_massa  885 non-null    float64
 5   severidade          961 non-null    object 
dtypes: float64(5), object(1)
memory usage: 45.2+ KB


## Valores nulos

In [4]:
print('Verificando os valores nulos: \n')
for col in dados.columns:
    print('Coluna '+col+': '+str(dados[col].isnull().sum()))

Verificando os valores nulos: 

Coluna bi_rads: 2
Coluna idade: 5
Coluna forma_da_massa: 31
Coluna contorno_da_massa: 48
Coluna densidade_da_massa: 76
Coluna severidade: 0


In [5]:
for col in dados.iloc[:, 0:5].columns:
    dados[col].fillna(method='ffill', inplace=True)

## Limpeza da base

### Dados ruidosos

In [6]:
dados['bi_rads'][dados[dados['bi_rads']==55.0].index[0]] = 5.0
dados['bi_rads'].value_counts()

4.0    549
5.0    346
3.0     36
2.0     14
6.0     11
0.0      5
Name: bi_rads, dtype: int64

### Padronização dos dados

In [7]:
for col in dados.iloc[:, 0:5].columns:
    dados[col] = dados[col].astype(int)    
dados.dtypes

bi_rads                int64
idade                  int64
forma_da_massa         int64
contorno_da_massa      int64
densidade_da_massa     int64
severidade            object
dtype: object

### Visualizando o resultado

In [8]:
dados.head()

,bi_rads,idade,forma_da_massa,contorno_da_massa,densidade_da_massa,severidade
0,5,67,3,5,3,maligno
1,4,43,1,1,3,maligno
2,5,58,4,5,3,maligno
3,4,28,1,1,3,benigno
4,5,74,1,5,3,maligno


## Estatística dos dados

In [9]:
dados.describe()

,bi_rads,idade,forma_da_massa,contorno_da_massa,densidade_da_massa
count,961.000000,961.000000,961.000000,961.000000,961.000000
mean,4.295525,55.406868,2.707596,2.783559,2.894901
std,0.705684,14.506748,1.244502,1.568233,0.406183
min,0.000000,18.000000,1.000000,1.000000,1.000000
25%,4.000000,45.000000,2.000000,1.000000,3.000000
50%,4.000000,57.000000,3.000000,3.000000,3.000000
75%,5.000000,66.000000,4.000000,4.000000,3.000000
max,6.000000,96.000000,4.000000,5.000000,4.000000


## Dados por classe

In [11]:
print('%dados por classe:')
round(dados['severidade'].value_counts(normalize=True)*100, 2)

%dados por classe:


benigno    53.69
maligno    46.31
Name: severidade, dtype: float64